In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from tqdm import tqdm
import pickle
import os

DATA_PATH = './data'

def plot_sum(df, group_col, title, xticks_map=None, typ='bar'):
    grp = df.groupby(group_col).size()
    x = [xticks_map[x] for x in grp.index] if xticks_map else grp.index
    if typ=='bar':
        plt.bar(x, grp.values)
    else:
        plt.plot(x, grp.values, typ)
    plt.xticks(x)
    plt.title(title)

In [ ]:
df_train = pd.read_csv(os.path.join(DATA_PATH,'wildfires_train.csv'),parse_dates=['date'],index_col='fire_id')
print(df_train.shape)

In [ ]:
df_train.head()

In [ ]:
fire_types = df_train.groupby('fire_type').fire_type_name.min().to_dict()
fire_types

**Даны координаты и время события, нужно отнести его к одному из 11 классов.**

## Распределение по классам

In [ ]:
rows_by_type = df_train.groupby('fire_type_name').size().sort_values()
plt.figure(figsize=(12,6))
plt.barh(rows_by_type.index, rows_by_type.values);

In [ ]:
print("Дисбаланс классов: min",rows_by_type.values.min()," max", rows_by_type.values.max())

## Время

In [ ]:
df_train['weekday'] = df_train.date.dt.weekday
df_train['month'] = df_train.date.dt.month
df_train['year'] = df_train.date.dt.year

print("Период: ",df_train.date.min().strftime("%d.%m.%Y"),"-",df_train.date.max().strftime("%d.%m.%Y"))

Данные за 2019 год не полные, нужно учитывать это при анализе распределений

### Распределение по месяцам

In [ ]:
plt.figure(figsize=(16,12))
for fire_type, title in fire_types.items():
    plt.subplot(3,4,fire_type)
    plot_sum(df_train[(df_train.year<2019) & (df_train.fire_type==fire_type)],'month',title);

plt.subplot(3,4,12)
plot_sum(df_train[(df_train.year<2019)],'month',"Итого по всем типам");

plt.suptitle('Распределение по месяцам для каждого типа событий');

- Зимой торфянники не горят.
- Лес горит весной и летом.
- Мусор жгут весной и осенью.
- Распределение ложных сигналов примерно соответствует распределению общего числа пожаров.

In [ ]:
plt.figure(figsize=(16,12))
for month,title in ((1,"Январь"), (2,"Февраль"), (3,"Март"), (4,"Апрель"), (5,"Май"), (6,"Июнь"),
                    (7,"Июль"), (8,"Август"), (9,"Сентябрь"), (10,"Октябрь"), (11,"Ноябрь"), (12,"Декабрь")):
    plt.subplot(3,4,month)
    plot_sum(df_train[df_train.month==month],'fire_type',title);

plt.suptitle('Распределение по типам событий в каждом месяце');

**Распределение событий по типам в каждом месяце сильно отличается.**

### Распределение по дням недели

In [ ]:
weekdays = {0:'Пн',1:'Вт',2:'Ср',3:'Чт',4:'Пт',5:'Сб',6:'Вс'}
plt.figure(figsize=(16,12))
for fire_type, title in fire_types.items():
    plt.subplot(3,4,fire_type)
    plot_sum(df_train[df_train.fire_type==fire_type],'weekday',title,weekdays);

plt.subplot(3,4,12)
plot_sum(df_train,'weekday',"Итого по всем типам",weekdays);

plt.suptitle('Распределение по дням недели для каждого типа событий');

**"Неконтролируемый пал" немного смещён к концу недели, "контролируемый" - к середине.**

**У некоторых типов событий заметна некоторая неравномерность по дням недели**

## Долгосрочная динамика

In [ ]:
year_map = {i:i-2000 for i in range(2012,2019)}
plt.figure(figsize=(16,12))
for fire_type, title in fire_types.items():
    plt.subplot(3,4,fire_type)
    plot_sum(df_train[(df_train.year<2019) & (df_train.fire_type==fire_type)], 'year', title, year_map, typ='-');

plt.subplot(3,4,12)
plot_sum(df_train[(df_train.year<2019)], 'year', 'Итого по всем типам', year_map, typ='-');

plt.suptitle('Распределение по годам для каждого типа событий');

In [ ]:
plt.figure(figsize=(16,12))
for fire_type in range(1,12):
    grp = df_train[(df_train.year<2019) & (df_train.fire_type==fire_type)].year.value_counts().sort_index()
    plt.plot(grp.index,grp.values,label=fire_types[fire_type]);
plt.legend();
plt.suptitle('Распределение событий по годам');

In [ ]:
plt.figure(figsize=(16,8))
for year in range(2012,2020):
    plt.subplot(2,4,year-2011)
    plot_sum(df_train[df_train.year==year],'fire_type',year if year<2019 else '2019 (4 месяца)');

plt.suptitle('Распределение по типам событий в каждый год');

**Распределение существенно меняется со временем.**

## География 

In [ ]:
print('Широта : %f - %f' %(df_train.latitude.min(),df_train.latitude.max()))
print('Долгота : %f - %f' %(df_train.longitude.min(),df_train.longitude.max()))

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
fig=plt.figure(figsize=(16,8))
m = Basemap(llcrnrlon=14,llcrnrlat=36,urcrnrlon=179,urcrnrlat=73,projection='merc')
m.bluemarble(scale=0.5)
m.drawcountries(color='yellow')
m.scatter(latlon=True, x=df_train.longitude.values, y=df_train.latitude.values,
          c=df_train.fire_type.values, s=1, cmap='hot', alpha=0.5)
plt.show()

In [ ]:
def plotmap(df, lon="longitude", lat="latitude", title='', color=None, size=None):
    m = Basemap(llcrnrlon=14,llcrnrlat=36,urcrnrlon=179,urcrnrlat=73,projection='merc')
    m.fillcontinents(color='#f0f0f0',zorder=0)
    m.drawcountries(color='#888888')
    m.scatter(latlon=True, x=df[lon].values, y=df[lat].values, color='blue',
              s=1 if size is None else df[size], alpha=0.5)
    plt.title(title)

In [ ]:
fig=plt.figure(figsize=(16,24))

for TYPE in range(1,12):
    plt.subplot(6,2,TYPE)
    plotmap(df_train[df_train.fire_type==TYPE],title=fire_types[TYPE])

plt.subplot(6,2,12)
plotmap(df_train,title="Итого по всем типам")

**Распределение событий по территории неоднородно и зависит от типа.**

## Интерактивные карты

(Работают только с plotly 4, в контейнере версия 3.3.)

In [ ]:
import plotly
if plotly.__version__>='4.0.0':
    from plotly import express as px
    def plotmap(df,lon='longitude',lat='latitude',title=''):
        fig = px.scatter_mapbox(df, lat=lat, lon=lon, color=color, size=size,
                          color_continuous_scale=px.colors.cyclical.IceFire)
        fig.update_layout(mapbox_style='carto-positron', mapbox_zoom=2, mapbox_center = {'lat': 60, 'lon': 60})
        fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
        print(title)
        fig.show()
    for TYPE in range(1,12):
        plotmap(df_train[df_train.fire_type==TYPE],title=fire_types[TYPE])